In [3]:
import importlib
from pprint import pprint

import pandas as pd
from cuery import search
from cuery.seo import geo
from cuery.utils import set_env

set_env(apify_secrets=False)

# LLM prompting with web search

In [12]:
model = "openai/gpt-4.1-mini"
# model = "google/gemini-2.5-flash"
# model = "xai/grok-3"
# model = "deepseek/deepseek-chat"

prompts = [
    "Mejores SUVs en España 2025 precio-calidad",
    "Cuáles son los mejores SUVs chinos dispnibles en España 2025",
]

responses = await search.query(prompts, model=model, max_concurrent=10, use_search=True)
responses.to_pandas()

Gathering searches:   0%|          | 0/2 [00:00<?, ?it/s]

prompt  \
0         Mejores SUVs en España 2025 precio-calidad   
1  Cuáles son los mejores SUVs chinos dispnibles ...   

                                                text  \
0  Si estás buscando los mejores SUVs en España p...   
1  En 2025, el mercado español ofrece una varieda...   

                                          references  
0  [{'title': 'Los 20 Mejores SUV de 2025 por Men...  
1  [{'title': 'Cinco SUV chinos que no tienen ya ...

# Competitor extraction

In [ ]:
importlib.reload(geo)

competitors = await geo.find_competitors(
    brands=["BYD"],
    sector="automotive",
    market="Spain",
    max_count=5,
)
competitors

In [ ]:
competitors = ["toyota", "kia", "mg", "renault", "omoda"]

In [ ]:
from cuery.seo import geo

importlib.reload(geo)

prompts = await geo.commercial_prompts(
    n=10,
    language="en",
    sector=None,  # "automotive",
    market=None,  # "Spain",
    brands=["BYD"] + competitors,
    include_brands="never",
)

pprint(prompts)

In [ ]:
importlib.reload(search)
importlib.reload(geo)

use_search = False
models = ["openai/gpt-4.1", "google/gemini-2.5-flash"]
# models = ["deepseek/deepseek-chat"]

df = await geo.query_with_models(prompts, models, use_search=use_search)
df

In [ ]:
df.to_parquet("geo_results.parquet", index=False)

In [ ]:
df = df[[col for col in df if not any(x in col for x in ("_ranks", "_pos"))]].copy()
df

In [ ]:
importlib.reload(geo)

df2 = geo.add_brand_ranks(
    df,
    brands=["Opel", "Dacia", "BYD", "Kia", "Renault", "MG", "Omoda", "Jaecoo"],
)
df2

# All-in-one

In [13]:
importlib.reload(geo)

params = {
    "brands": ["Peugeot", "Opel"],
    "models": ["openai/gpt-4.1", "google/gemini-2.5-flash", "deepseek/deepseek-chat"],
    "prompts": ["best electric SUVs for families in 2024"],
    "prompts_max": 10,
    "prompt_language": "English",
    "prompt_intents": ["commercial", "transactional"],
    "brands_in_prompt": "never",
    "competitors": ["BYD"],
    "competitors_max": 10,
    "sector": "automotive",
    "market": "Spain",
    "use_search": True,
}

cfg = geo.GeoConfig(**params)

In [14]:
result = await geo.analyse(cfg)
result

2025-09-12 15:24:16 [cuery|INFO] Querying LLMs with

GeoConfig
—————————
{
  "brands": [
    "peugeot",
    "opel"
  ],
  "models": [
    "openai/gpt-4.1",
    "google/gemini-2.5-flash",
    "deepseek/deepseek-chat"
  ],
  "prompts": [
    "best electric SUVs for families in 2024"
  ],
  "prompts_max": 10,
  "prompt_intents": [
    "commercial",
    "transactional"
  ],
  "prompt_language": "English",
  "brands_in_prompt": "never",
  "competitors": [
    "BYD"
  ],
  "competitors_max": 10,
  "competitors_model": "openai/gpt-4.1",
  "sector": "automotive",
  "market": "Spain",
  "use_search": true
}

2025-09-12 15:24:16 [cuery|INFO] Using these seed brands: ['opel', 'peugeot']
2025-09-12 15:24:16 [cuery|INFO] Searching for competitor brands with prompt:
Identify up to 10 real competing brands for ['opel', 'peugeot'] in the 'automotive' sector in the 'Spain' market


Gathering searches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-09-12 15:24:24 [cuery|INFO] Extracting competitor names from search result
2025-09-12 15:24:27 [cuery|INFO] Found these competitors brands: ['renault', 'omoda', 'volkswagen', 'seat', 'byd', 'mg', 'toyota', 'kia', 'dacia', 'ford']
2025-09-12 15:24:27 [cuery|INFO] Generating search queries with prompt:
"Generate 9 unique, concise LLM prompts with one or more of the following intents: ['commercial', 'transactional']. Cover realistic user intentions like comparisons, alternatives, trust/regulatory, location specific queries etc. The prompts should be similar to Google search queries but adapted to how users would ask an LLM. Generate prompts in the 'English' language. Focus on the 'automotive' sector. Focus on the 'Spain' market. If brand context helps, consider these brands: ['opel', 'peugeot', 'renault', 'omoda', 'volkswagen', 'seat', 'byd', 'mg', 'toyota', 'kia', 'dacia', 'ford']. Do not mention any brand names explicitly in the queries though. Strictly return a JSON array of strin

Gathering searches:   0%|          | 0/30 [00:00<?, ?it/s]

2025-09-12 15:25:56 [cuery|INFO] Analysing brand ranks
2025-09-12 15:25:57 [cuery|INFO] Summarising own brand and competitor ranks
2025-09-12 15:25:57 [cuery|INFO] Got results dataframe:
                                              prompt  own_in_txt_count  \
0  Are there any trade-in deals for upgrading to ...                 0   
1  Compare hybrid compact cars available in Spain...                 1   
2  Find certified used cars with warranty near Ba...                 0   
3  Find safe petrol cars for long-distance travel...                 1   
4  Show me 7-seater cars with the best resale val...                 1   
5  Show me affordable hatchback lease options in ...                 2   
6  What are the financing options for new sedans ...                 2   
7  What government incentives exist for buying ec...                 0   
8  Which vehicles have the lowest insurance costs...                 0   
9            best electric SUVs for families in 2024                 0   

prompt  own_in_txt_count  \
0  Are there any trade-in deals for upgrading to ...                 0   
1  Compare hybrid compact cars available in Spain...                 1   
2  Find certified used cars with warranty near Ba...                 0   
3  Find safe petrol cars for long-distance travel...                 1   
4  Show me 7-seater cars with the best resale val...                 1   
5  Show me affordable hatchback lease options in ...                 2   
6  What are the financing options for new sedans ...                 2   
7  What government incentives exist for buying ec...                 0   
8  Which vehicles have the lowest insurance costs...                 0   
9            best electric SUVs for families in 2024                 0   

   own_in_refs_count  cmp_in_txt_count  cmp_in_refs_count own_in_txt_gpt_4_1  \
0                  0                 0                  0                  ❌   
1                  0                 3                  2                  ✅   
2                  0                 3                  2                  ❌   
3                  0                 3                  0                  ❌   
4                  0                 3                  0                  ❌   
5                  0                 3                  1                  ✅   
6                  1                 3                  1                  ❌   
7                  0                 1                  1                  ❌   
8                  0                 2                  0                  ❌   
9                  0                 3                  2                  ❌   

   own_in_txt_pos_gpt_4_1 own_in_refs_gpt_4_1 own_in_refs_pos_gpt_4_1  \
0                     NaN                   ❌                    None   
1                     3.0                   ❌                    None   
2                     NaN                   ❌                    None   
3                     NaN                   ❌                    None   
4                     NaN                   ❌                    None   
5                     4.0                   ❌                    None   
6                     NaN                   ❌                    None   
7                     NaN                   ❌                    None   
8                     NaN                   ❌                    None   
9                     NaN                   ❌                    None   

  own_in_txt_gemini_2_5_flash  ...  \
0                           ❌  ...   
1                           ❌  ...   
2                           ❌  ...   
3                           ❌  ...   
4                           ✅  ...   
5                           ✅  ...   
6                           ✅  ...   
7                           ❌  ...   
8                           ❌  ...   
9                           ❌  ...   

                                        text_gpt_4_1 references_deepseek_chat  \
0  Here’s an overview of **current trade-in and u...                       []   
1  Below is a detailed comparison of popular hybr...                       []   
2  Here are several dealerships and programs offe...                       []   
3  Here are some excellent petrol-powered cars su...                       []   
4  Here are some of the **7‑seater vehicles** ava...                       []   
5  Here’s a structured overview of affordable hat...                       []   
6  Here is an in-depth overview of financing opti...                       []   
7  Here’s a comprehensive overview of current gov...                       []   
8  Here’s what you need to know about the vehicle...                       []   
9  Here are some of the **best electric SUVs for ...                       []   

                         references_gemini_2_5_flash  \
0  [{'title': 'apple.com', 'url': 'https://www.ap...   
1  [{'title': 'autogiz.com', 'url': 'https://www....   
2  [{'title': 'tresdarc.com', 'url': 'https://tre...   
3  [{'title

In [15]:
result.to_parquet("geo_results.parquet", index=False)

In [16]:
result = pd.read_parquet("geo_results.parquet")
result

prompt  own_in_txt_count  \
0  Are there any trade-in deals for upgrading to ...                 0   
1  Compare hybrid compact cars available in Spain...                 1   
2  Find certified used cars with warranty near Ba...                 0   
3  Find safe petrol cars for long-distance travel...                 1   
4  Show me 7-seater cars with the best resale val...                 1   
5  Show me affordable hatchback lease options in ...                 2   
6  What are the financing options for new sedans ...                 2   
7  What government incentives exist for buying ec...                 0   
8  Which vehicles have the lowest insurance costs...                 0   
9            best electric SUVs for families in 2024                 0   

   own_in_refs_count  cmp_in_txt_count  cmp_in_refs_count own_in_txt_gpt_4_1  \
0                  0                 0                  0                  ❌   
1                  0                 3                  2                  ✅   
2                  0                 3                  2                  ❌   
3                  0                 3                  0                  ❌   
4                  0                 3                  0                  ❌   
5                  0                 3                  1                  ✅   
6                  1                 3                  1                  ❌   
7                  0                 1                  1                  ❌   
8                  0                 2                  0                  ❌   
9                  0                 3                  2                  ❌   

   own_in_txt_pos_gpt_4_1 own_in_refs_gpt_4_1 own_in_refs_pos_gpt_4_1  \
0                     NaN                   ❌                    None   
1                     3.0                   ❌                    None   
2                     NaN                   ❌                    None   
3                     NaN                   ❌                    None   
4                     NaN                   ❌                    None   
5                     4.0                   ❌                    None   
6                     NaN                   ❌                    None   
7                     NaN                   ❌                    None   
8                     NaN                   ❌                    None   
9                     NaN                   ❌                    None   

  own_in_txt_gemini_2_5_flash  ...  \
0                           ❌  ...   
1                           ❌  ...   
2                           ❌  ...   
3                           ❌  ...   
4                           ✅  ...   
5                           ✅  ...   
6                           ✅  ...   
7                           ❌  ...   
8                           ❌  ...   
9                           ❌  ...   

                                        text_gpt_4_1 references_deepseek_chat  \
0  Here’s an overview of **current trade-in and u...                       []   
1  Below is a detailed comparison of popular hybr...                       []   
2  Here are several dealerships and programs offe...                       []   
3  Here are some excellent petrol-powered cars su...                       []   
4  Here are some of the **7‑seater vehicles** ava...                       []   
5  Here’s a structured overview of affordable hat...                       []   
6  Here is an in-depth overview of financing opti...                       []   
7  Here’s a comprehensive overview of current gov...                       []   
8  Here’s what you need to know about the vehicle...                       []   
9  Here are some of the **best electric SUVs for ...                       []   

                         references_gemini_2_5_flash  \
0  [{'title': 'apple.com', 'url': 'https://www.ap...   
1  [{'title': 'autogiz.com', 'url': 'https://www....   
2  [{'title': 'tresdarc.com', 'url': 'https://tre...   
3  [{'title

In [5]:
importlib.reload(geo)

own = ["Peugeot"]
competitors = [
    "citroën",
    "mercedes-benz",
    "volkswagen",
    "mg",
    "dacia",
    "renault",
    "toyota",
    "hyundai",
    "seat",
    "kia",
]

geo.summarize_ranks(result, models=cfg.models, own=own, competitors=competitors, emoji_flags=True)

NameError: name 'cfg' is not defined

In [1]:
df = result
df

NameError: name 'result' is not defined

In [41]:
importlib.reload(geo)

result = result[[col for col in result if not any(x in col for x in ("_ranks", "_pos"))]].copy()
result = geo.add_brand_ranks(
    result,
    brands=cfg.brands_seed
    + [
        "citroën",
        "mercedes-benz",
        "volkswagen",
        "mg",
        "dacia",
        "renault",
        "toyota",
        "hyundai",
        "seat",
        "kia",
    ],
)
result

prompt  \
0        approved used cars warranty Spain   
1  best electric SUVs for families in 2024   
2              best suv offers Madrid 2024   
3                 car safety ratings Spain   
4          compact crossover reviews Spain   
5      compare hybrid vs diesel cars Spain   
6   eco-friendly city cars available Spain   
7        electric car tax incentives Spain   
8      family hatchback alternatives Spain   
9           lease vs finance new car Spain   

                         text_deepseek/deepseek-chat  \
0  I understand you're looking for information ab...   
1  Based on my research, here are the best electr...   
2  I understand you're looking for the best SUV o...   
3  I understand you're looking for car safety rat...   
4  I understand you're looking for compact crosso...   
5  ## Hybrid vs Diesel Cars in Spain: Comparison\...   
6  Based on your query about eco-friendly city ca...   
7  ## Electric Car Tax Incentives in Spain\n\n###...   
8  Based on your query about family hatchback alt...   
9  When deciding between leasing and financing a ...   

                        text_google/gemini-2.5-flash  \
0  In Spain, the warranty for approved used cars ...   
1  For families seeking an eco-friendly and pract...   
2  While specific "best" offers can be subjective...   
3  Car safety ratings in Spain are primarily dete...   
4  In the Spanish market, compact crossovers cont...   
5  ## Hybrid vs. Diesel Cars in Spain: A Comprehe...   
6  Spain offers a growing selection of eco-friend...   
7  Spain offers several tax incentives and subsid...   
8  For families in Spain looking for alternatives...   
9  When considering a new car in Spain, both leas...   

                                 text_openai/gpt-4.1  \
0  If you're looking into warranties for "approve...   
1  Here’s a well-rounded overview of the best ele...   
2  Aquí tienes algunas de las **mejores ofertas d...   
3  Aquí tienes una visión clara y actualizada sob...   
4  Aquí tienes una visión actualizada (septiembre...   
5  Here’s a thorough comparison of hybrid versus ...   
6  Here’s a comprehensive overview of eco‑friendl...   
7  Here is an up-to-date overview (as of Septembe...   
8  If you're looking for practical alternatives t...   
9  Aquí tienes una comparativa clara y detallada ...   

  references_deepseek/deepseek-chat  \
0                                []   
1                                []   
2                                []   
3                                []   
4                                []   
5                                []   
6                                []   
7                                []   
8                                []   
9                                []   

                  references_google/gemini-2.5-flash  \
0  [{'title': 'mylawyerinspain.com', 'url': 'http...   
1  [{'title': 'kbb.com', 'url': 'https://www.kbb....   
2  [{'title': 'coches.net', 'url': 'https://www.c...   
3  [{'title': 'byd.com', 'url': 'https://www.byd....   
4  [{'title': 'youtube.com', 'url': 'https://www....   
5  [{'title': 'globalpetrolprices.com', 'url': 'h...   
6  [{'title': 'euroweeklynews.com', 'url': 'https...   
7  [{'title': 'electrive.com', 'url': 'https://ww...   
8  [{'title': '6wresearch.com', 'url': 'https://v...   
9  [{'title': 'autoleaserenting.es', 'url': 'http...   

                           references_openai/gpt-4.1  \
0  [{'title': '! Spanish News Today - Extended Wa...   
1  [{'title': 'Edmunds Top Rated Awards for 2024 ...   
2  [{'title': 'BYD llega a Grupo Gamboa: descubre...   
3  [{'title': 'DGT - Clasificación de seguridad d...   
4  [{'title': 'Citroën C3 Aircross, Coche del Año...   
5  [{'title': 'Car sales in Spain by energy sourc...   
6  [{'title': 'Best Small Electric Car 2025: Pric...   
7  [{'title': 'Spain extends Moves III programme ...   
8  [{'title': 'Top 5 Station Wagons to Buy in 202...   
9  [{'title': 'Leasing vs Renting de Coches en Es...   

              